<h1>COMP6940: Assignment 2</h1>
<p>Kevan Lee Lum 816003573</p>
<p>Due date: March 18 2018</p>
<p>Windows: pyspark session started with pyspark --master local[2]</p>
<p>Ubuntu: For some reason I need to set export PYSPARK_PYTHON=/usr/bin/python3 and export PYSPARK_DRIVER_PYTHON=python3 every time</p>

<h2 style="border-top:2px solid black; border-bottom:2px solid black; padding:10px 0px 10px 0px; text-align:center;">Task A</h2>
<p>1. Convert the following sentence into a python tuple list of letters and the frequency of which each letter appears in the current word. Ignore all non-alpha numeric characters.</p>

In [1]:
sentence = "The quick brown fox jumps over the lazy dog and the fox was very happy"

tuplelist = []

We can convert all to lowercase to make things easier

In [2]:
lowersentence = sentence.lower()

<p>Add a whitespace at the end for fun. Actually its so that the last word is not forgotten. The whitespaces are used to determine the end of the word.</p>

In [3]:
lowersentence += " "

In [4]:
currentworddict = {}
for letter in lowersentence:
    if letter == " ":
        for key in currentworddict.keys():
            tuplelist.append((key, currentworddict[key]))
        currentworddict = {}
    else:
        if letter in currentworddict.keys():
            currentworddict[letter] += 1
        else:
            currentworddict[letter] = 1
                             
print(tuplelist)

[('t', 1), ('e', 1), ('h', 1), ('i', 1), ('c', 1), ('u', 1), ('q', 1), ('k', 1), ('b', 1), ('o', 1), ('n', 1), ('r', 1), ('w', 1), ('o', 1), ('x', 1), ('f', 1), ('m', 1), ('s', 1), ('u', 1), ('j', 1), ('p', 1), ('o', 1), ('r', 1), ('v', 1), ('e', 1), ('t', 1), ('e', 1), ('h', 1), ('a', 1), ('l', 1), ('z', 1), ('y', 1), ('o', 1), ('d', 1), ('g', 1), ('n', 1), ('a', 1), ('d', 1), ('t', 1), ('e', 1), ('h', 1), ('o', 1), ('x', 1), ('f', 1), ('a', 1), ('s', 1), ('w', 1), ('y', 1), ('r', 1), ('v', 1), ('e', 1), ('h', 1), ('a', 1), ('y', 1), ('p', 2)]


<p>Well that was fun. If there was a comma or period or something other than a letter in the sentence, the if statement could have read if letter not in some list of valid characters, do something.</p>
<p>2. Create a PySpark Context</p>
<p>What's a PySpark Context? From the docs, its the main entry point for Spark functionality, which represents the connection to a Spark cluster, and can be used to create RDD and broadcast variables on that cluster.</p>

In [5]:
import pyspark

sc = pyspark.SparkContext.getOrCreate()

<p>3. Convert the list of tuples into a PySpark RDD. RDD is a Resilient Distributed Dataset whch are dstributed collections of objects that can be cached in memory across cluster nodes. It represents an immutable, partitioned collection of elements that can be operated on in parallel. It will be interesting to compare the timing of these operations to those done without Spark.</p> 

<p>The parallelize function takes a Python structure to form an RDD. It is alledged</p>

In [6]:
rdd = sc.parallelize(tuplelist)

<p>4. Using the methods of PySpark RDD display the letter count.</p>
<p>The reduceByKey() method merges the values for each key using an associative and cummutative reduce function. In this case they are added.  The countByKey and reduceByKey methods also procude similar results but return dictionaries.</p>

In [7]:
from operator import add
rdd.reduceByKey(add).sortByKey().collect()

[('a', 4),
 ('b', 1),
 ('c', 1),
 ('d', 2),
 ('e', 5),
 ('f', 2),
 ('g', 1),
 ('h', 4),
 ('i', 1),
 ('j', 1),
 ('k', 1),
 ('l', 1),
 ('m', 1),
 ('n', 2),
 ('o', 5),
 ('p', 3),
 ('q', 1),
 ('r', 3),
 ('s', 2),
 ('t', 3),
 ('u', 2),
 ('v', 2),
 ('w', 2),
 ('x', 2),
 ('y', 3),
 ('z', 1)]

<p>5. Using the methods of PySpark RDD display the letter and number of times they appear in each word in the sentence.</p>
<p>The groupByKey() method groups the values for each key in the RDD into a single sequence. Nice.</p>

In [8]:
sorted(rdd.groupByKey().mapValues(list).collect())

[('a', [1, 1, 1, 1]),
 ('b', [1]),
 ('c', [1]),
 ('d', [1, 1]),
 ('e', [1, 1, 1, 1, 1]),
 ('f', [1, 1]),
 ('g', [1]),
 ('h', [1, 1, 1, 1]),
 ('i', [1]),
 ('j', [1]),
 ('k', [1]),
 ('l', [1]),
 ('m', [1]),
 ('n', [1, 1]),
 ('o', [1, 1, 1, 1, 1]),
 ('p', [1, 2]),
 ('q', [1]),
 ('r', [1, 1, 1]),
 ('s', [1, 1]),
 ('t', [1, 1, 1]),
 ('u', [1, 1]),
 ('v', [1, 1]),
 ('w', [1, 1]),
 ('x', [1, 1]),
 ('y', [1, 1, 1]),
 ('z', [1])]

<h2 style="border-top:2px solid black; border-bottom:2px solid black; padding:10px 0px 10px 0px; text-align:center;">Task B</h2>
<p>1. Create a sql context from PySpark SQLContext.</p>

In [9]:
from pyspark.sql import Row, SparkSession

In [10]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Task 2") \
    .config("spark.executor.memory", "1gb") \
    .getOrCreate()
    
sc2 = spark.sparkContext

<p>2. Load the Amazon Review Dataset into a PySpark RDD, ensure that each row is properly separated and the headers are matched to their respective columns.</p>

In [11]:
rdd2 = sc2.textFile('amazon_reviews.csv')

In [12]:
rdd2.take(3)

[',id,name,username',
 '0,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Cristina M',
 '1,AVpe7AsMilAPnD_xQ78G,Kindle Paperwhite,Ricky']

<p>Oh problems. Each row appears to be one long string instead of separate columns. Since it is a csv, we split the columns at the "," using the map function.</p>

In [13]:
rdd2 = rdd2.map(lambda line: line.split(","))

In [14]:
rdd2.take(3)

[['', 'id', 'name', 'username'],
 ['0', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Cristina M'],
 ['1', 'AVpe7AsMilAPnD_xQ78G', 'Kindle Paperwhite', 'Ricky']]

<p>Looks better now, although the lack of name on the first column might cause some problems</p>
<p>3. Convert the rdd into a PySpark DataFrame.</p>

In [15]:
df = rdd2.map(lambda line: Row(no=line[0], 
                              id=line[1], 
                              product=line[2],
                              username=line[3])).toDF()

df.head(5)

[Row(id='id', no='', product='name', username='username'),
 Row(id='AVpe7AsMilAPnD_xQ78G', no='0', product='Kindle Paperwhite', username='Cristina M'),
 Row(id='AVpe7AsMilAPnD_xQ78G', no='1', product='Kindle Paperwhite', username='Ricky'),
 Row(id='AVpe7AsMilAPnD_xQ78G', no='2', product='Kindle Paperwhite', username='Tedd Gardiner'),
 Row(id='AVpe7AsMilAPnD_xQ78G', no='3', product='Kindle Paperwhite', username='Dougal')]

<p>Looks good. The no. column appears to be a string but that's probably fine.</p>
<p>4. Using the dataframe from question 3 show the top 20 bought products.</p>
<p>The groupBy() method groups the DF using the 'name' column, to allow the use of the count agg function. The resulting DF is then sorted in descending order and the 20 most popular items are extracted using show(). The rows are not truncated by using False in the show method.</p>

In [16]:
df.groupBy(['product']).count().sort("count", ascending=False).show(20, False)

+-----------------------------------------------------------------------------------------+-----+
|product                                                                                  |count|
+-----------------------------------------------------------------------------------------+-----+
|Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker                                    |542  |
|Amazon Fire TV                                                                           |166  |
|Amazon Premium Headphones                                                                |133  |
|Fire HD 6 Tablet                                                                         |87   |
|"Kindle Fire HDX 7"""                                                                    |53   |
|"Kindle Fire HDX 8.9"""                                                                  |43   |
|"Kindle Fire HD 7"""                                                                     |41   |
|Kindle Paperwhite  

<p>5. Using the dataframe from question 3  show the top 20 users and the products that they purchased.</p>
<p>Can use the previous code to determine the 20 most frequent users</p>

In [17]:
df.groupBy(['username','product']).count().sort("count", ascending=False).show(20, False)

+-------------------+----------------------------------------------------------------+-----+
|username           |product                                                         |count|
+-------------------+----------------------------------------------------------------+-----+
|A. Younan          |Amazon Premium Headphones                                       |59   |
|Andrew             |Amazon Premium Headphones                                       |43   |
|Victor L.          |Amazon Premium Headphones                                       |30   |
|William Hardin     |Fire HD 6 Tablet                                                |30   |
|Mike W.            |Fire HD 6 Tablet                                                |29   |
|Earthling1984      |Fire HD 6 Tablet                                                |28   |
|B. Tarbuck         |"Kindle Fire HDX 8.9"""                                         |16   |
|William Hardin     |Amazon Fire TV                                   

<p>These answers are different from those in question 9. The answers here refer to people who have bought a single propduct multiple times. It possibly depends on how the question is interpreted. Oh well.</p>

<p>6. Create a RDD of tuples from the dataframe from question 3 with only 2 columns ‘product’  and ‘username’ in that order.</p>

In [18]:
rdd3 = df['product', 'username'].rdd

rdd3.take(3)

[Row(product='name', username='username'),
 Row(product='Kindle Paperwhite', username='Cristina M'),
 Row(product='Kindle Paperwhite', username='Ricky')]

<p>7. Using methods from PySpark’s RDD object e.g. groupByKey, map, reduceByKey, derive the top 20 products.</p>
<p>Assuming this would give the same results as question 4</p>

In [19]:
rdd3.groupByKey().mapValues(len).sortBy(lambda x: x[1], ascending=False).take(20)

[('Amazon Tap - Alexa-Enabled Portable Bluetooth Speaker', 542),
 ('Amazon Fire TV', 166),
 ('Amazon Premium Headphones', 133),
 ('Fire HD 6 Tablet', 87),
 ('"Kindle Fire HDX 7"""', 53),
 ('"Kindle Fire HDX 8.9"""', 43),
 ('"Kindle Fire HD 7"""', 41),
 ('Kindle Paperwhite', 39),
 ('Certified Refurbished Amazon Fire TV (Previous Generation - 1st)', 38),
 ('Kindle Keyboard', 32),
 ('All-New Amazon Fire 7 Tablet Case (7th Generation', 27),
 ('Kindle', 20),
 ('Amazon 5W USB Official OEM Charger and Power Adapter for Fire Tablets and Kindle eReaders',
  19),
 ('All-New Amazon Fire HD 8 Tablet Case (7th Generation', 18),
 ('Replacement Remote for Amazon Fire TV Stick', 17),
 ('All-New Amazon Kid-Proof Case for Amazon Fire 7 Tablet (7th Generation',
  13),
 ('Echo Dot (2nd Generation) - Black', 13),
 ('Alexa Voice Remote for Amazon Fire TV and Fire TV Stick', 12),
 ('Moshi Anti-Glare No Bubble Screen Protector for the Fire Phone', 12),
 ('Amazon Kindle Oasis Premium Leather Battery Cover - Wa

<p>Spent alot of time trying to make countByKey() work there.</p>
<p>8. Create another RDD of tuples from the dataframe from question 3 with the columns ‘username’ and ‘product’ in that order.</p>

In [20]:
rdd4 = df['username', 'product'].rdd

rdd4.take(3)

[Row(username='username', product='name'),
 Row(username='Cristina M', product='Kindle Paperwhite'),
 Row(username='Ricky', product='Kindle Paperwhite')]

<p>9. Using methods from PySparks’s RDD object, produce the top 10 customers who purchased the most items. The top 10 list must show the usernames and a list of all the items each person bought. Each item should have an associated quantity value representing the amount of the item which was purchased by the customer.</p>
<p>Create a custom mapping function to apply to each value</p>

In [21]:
def f(x):
    y = list(x)
    z = {}
    for item in y:
        if item in z.keys():
            z[item] += 1
        else:
            z[item] = 1
    return (z, len(x))

rdd4.groupByKey().mapValues(f).sortBy(lambda x: x[1][1], ascending=False).take(10)


[('A. Younan', ({'Amazon Premium Headphones': 59}, 59)),
 ('William Hardin',
  ({'Amazon Fire TV': 16,
    'Certified Refurbished Amazon Fire TV (Previous Generation - 1st)': 12,
    'Fire HD 6 Tablet': 30},
   58)),
 ('Andrew', ({'Amazon Premium Headphones': 43}, 43)),
 ('Victor L.', ({'Amazon Premium Headphones': 30}, 30)),
 ('NF', ({'"Kindle Fire HD 7"""': 14, '"Kindle Fire HDX 7"""': 15}, 29)),
 ('Mike W.', ({'Fire HD 6 Tablet': 29}, 29)),
 ('Earthling1984', ({'Fire HD 6 Tablet': 28}, 28)),
 ('Amazon Reviewer',
  ({'"Kindle Fire HDX 7"""': 13, '"Kindle Fire HDX 8.9"""': 14}, 27)),
 ('Mandy',
  ({'Amazon Fire TV': 16,
    'Certified Refurbished Amazon Fire TV (Previous Generation - 1st)': 10},
   26)),
 ('Amazon Customer',
  ({'"Kindle Fire HD 7"""': 1,
    '"Kindle Fire HDX 8.9"""': 1,
    'Alexa Voice Remote for Amazon Fire TV and Fire TV Stick': 3,
    'All-New Amazon Fire 7 Tablet Case (7th Generation': 2,
    'All-New Amazon Kid-Proof Case for Amazon Fire 7 Tablet (7th Generati

<p>I'm not sure if I did that the intended way. I know that the rdd had to be grouped by keys in order for the customers to be ordered. And I know that map Values had to be used to return the tuple of what was bought and the total count.</p>